# **Assignment - 3**

Q1: K-Fold Cross Validation for Multiple Linear Regression (Least Square Error Fit)  
Download the dataset regarding USA House Price Prediction from the following link:  
https://drive.google.com/file/d/1O_NwpJT-8xGfU_-3llUl2sgPu0xllOrX/view?usp=sharing  
Load the dataset and Implement 5- fold cross validation for multiple linear regression  
(using least square error fit).  
Steps:  
  
a) Divide the dataset into input features (all columns except price) and output variable
(price) 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

df = pd.read_csv("Data/USA_Housing.csv")

X = df.drop("Price", axis=1).values
y = df["Price"].values


b) Scale the values of input features. 

In [2]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

c) Divide input and output features into five folds. 

In [3]:
n = len(X)
indices = np.arange(n)
np.random.shuffle(indices)  
folds = np.array_split(indices, 5)

d) Run five iterations, in each iteration consider one-fold as test set and remaining four sets as training set.   
Find the beta (𝛽) matrix, predicted values, and R2_score
for each iteration using least square error fit. 

In [6]:
def add_intercept(X):
    return np.hstack([np.ones((X.shape[0], 1)), X])

def least_squares_beta(X, y):
    # Normal equation: β = (X^T X)^(-1) X^T y
    return np.linalg.pinv(X.T @ X) @ X.T @ y

best_beta = None
best_r2 = -np.inf

for i in range(5):
    test_idx = folds[i]
    train_idx = np.hstack([folds[j] for j in range(5) if j != i])

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]

    X_train_int = add_intercept(X_train)
    X_test_int = add_intercept(X_test)

    beta = least_squares_beta(X_train_int, y_train)

    y_pred = X_test_int @ beta

    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    print(f"\nFold {i+1}")
    print("\nBeta (coefficients):")
    print(beta) 
    print("\nFirst 5 predictions:", y_pred[:5])
    print("\nFirst 5 actuals:    ", y_test[:5])
    print(f"R² = {r2:.4f}, MSE = {mse:.4f}")

    if r2 > best_r2:
        best_r2 = r2
        best_beta = beta



Fold 1

Beta (coefficients):
[1232183.14157037  230860.82977106  163813.31169062  123159.24698459
    1862.78381785  150779.51690422]

First 5 predictions: [1308929.94118012 1791050.82771068 1757636.94449517 1247747.29551711
  908104.4682454 ]

First 5 actuals:     [1499153.081 1833208.579 1741052.96  1153135.22   999244.635]
R² = 0.9203, MSE = 10073301346.7909

Fold 2

Beta (coefficients):
[1231704.18118797  229631.56013204  162896.37725785  120237.85792513
    2636.42063697  150492.62173255]

First 5 predictions: [ 975567.0208538  1010998.88461966  788964.54134108  686683.19769761
 2057672.48086813]

First 5 actuals:     [ 980161.6274  923246.8621  734562.1045  482689.7034 2219724.162 ]
R² = 0.9176, MSE = 10326721192.2157

Fold 3

Beta (coefficients):
[1233195.94576496  230422.33489483  164251.10231764  121489.18266983
    1243.82302154  150714.30346787]

First 5 predictions: [1279653.62019732 1454418.16407668 2036296.96870113 1022160.98837747
  886440.91620051]

First 5 actuals:   

e) Use the best value of (𝛽) matrix (for which R2_score is maximum), to train the
regressor for 70% of data and test the performance for remaining 30% data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_int = add_intercept(X_train)
X_test_int = add_intercept(X_test)

y_pred = X_test_int @ best_beta

print("\nUsing best β on 70/30 split:")
print("R² =", r2_score(y_test, y_pred))
print("MSE =", mean_squared_error(y_test, y_pred))



Using best β on 70/30 split:
R² = 0.9147509985464375
MSE = 10060262294.327486


Q2: Concept of Validation set for Multiple Linear Regression (Gradient Descent Optimization)  
Consider the same dataset of Q1, rather than dividing the dataset into five folds, divide the dataset into training set (56%), validation set (14%), and test set (30%).  
Consider four different values of learning rate i.e. {0.001,0.01,0.1,1}. Compute the values of regression coefficients for each value of learning rate after 1000 iterations.  
For each set of regression coefficients, compute R2_score for validation and test set and find the best value of regression coefficients.   

In [9]:
from sklearn.model_selection import train_test_split

# 70% train+val and 30% test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# temp into 80% train, 20% val (56% train, 14% val overall)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

X_train_int = add_intercept(X_train)
X_val_int = add_intercept(X_val)
X_test_int = add_intercept(X_test)


def gradient_descent(X, y, lr, iterations=1000):
    m, n = X.shape
    beta = np.zeros(n)
    for _ in range(iterations):
        preds = X @ beta
        error = preds - y
        grad = (1/m) * (X.T @ error)
        beta -= lr * grad
    return beta


learning_rates = [0.001, 0.01, 0.1, 1]

for lr in learning_rates:
    beta = gradient_descent(X_train_int, y_train, lr=lr, iterations=1000)

    y_val_pred = X_val_int @ beta
    y_test_pred = X_test_int @ beta

    print(f"lr={lr} -> R²_val={r2_score(y_val, y_val_pred):.4f}, R²_test={r2_score(y_test, y_test_pred):.4f}")



lr=0.001 -> R²_val=-0.8125, R²_test=-0.9914
lr=0.01 -> R²_val=0.9098, R²_test=0.9147
lr=0.1 -> R²_val=0.9098, R²_test=0.9148
lr=1 -> R²_val=0.9098, R²_test=0.9148


Q3: Pre-processing and Multiple Linear Regression  
Download the dataset regarding Car Price Prediction from the following link:    
https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data  

1. Load the dataset with following column names ["symboling", "normalized_losses",
"make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels",
"engine_location", "wheel_base", "length", "width", "height", "curb_weight",
"engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke",
"compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]
and replace all ? values with NaN 

In [18]:
colnames = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration","num_doors",
"body_style", "drive_wheels", "engine_location", "wheel_base", "length", "width",
"height", "curb_weight", "engine_type", "num_cylinders", "engine_size", "fuel_system",
"bore", "stroke", "compression_ratio", "horsepower", "peak_rpm", "city_mpg",
"highway_mpg", "price"]

df = pd.read_csv("Data/imports-85.txt", names=colnames)
df.replace("?", np.nan, inplace=True)


2. Replace all NaN values with central tendency imputation. Drop the rows with NaN
values in price column 

In [21]:
numeric_cols = ["symboling","normalized_losses","wheel_base","length","width","height","curb_weight",
                "engine_size","bore","stroke","compression_ratio","horsepower","peak_rpm",
                "city_mpg","highway_mpg","price"]

categorical_cols = ["make", "fuel_type", "aspiration","num_doors", "body_style", 
                    "drive_wheels", "engine_location", "engine_type", 
                    "num_cylinders", "fuel_system"]


for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Drop rows with missing price
df = df.dropna(subset=["price"])

# Fill NaNs: numeric cols -> mean
for col in numeric_cols:
    if df[col].isna().sum() > 0:
        df[col] = df[col].fillna(df[col].mean())

# Fill NaNs: categorical cols -> mode
for col in categorical_cols:
    if df[col].isna().sum() > 0:
        df[col] = df[col].fillna(df[col].mode()[0])


3. There are 10 columns in the dataset with non-numeric values. Convert these values to  
numeric values using following scheme:  
(i) For “num_doors” and “num_cylinders”: convert words (number names) to figures for e.g., two to 2    
(ii) For "body_style", "drive_wheels": use dummy encoding scheme  
(iii) For “make”, “aspiration”, “engine_location”,fuel_type: use label encoding scheme    
(iv) For fuel_system: replace values containing string pfi to 1 else all values to 0.  
(v) For engine_type: replace values containing string ohc to 1 else all values to 0.   

In [22]:
from sklearn.preprocessing import LabelEncoder

# Convert word numbers to digits
mapping = {"two":2, "four":4, "six":6, "eight":8, "twelve":12}
df["num_doors"] = df["num_doors"].map(mapping).fillna(4).astype(int)
df["num_cylinders"] = df["num_cylinders"].map(mapping).fillna(4).astype(int)

# Dummy encoding
df = pd.get_dummies(df, columns=["body_style","drive_wheels"], drop_first=True)

# Label encoding
for col in ["make","aspiration","engine_location","fuel_type"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Fuel system -> binary
df["fuel_system"] = df["fuel_system"].str.contains("pfi").astype(int)

# Engine type -> binary
df["engine_type"] = df["engine_type"].str.contains("ohc").astype(int)


4. Divide the dataset into input features (all columns except price) and output variable (price).  
Scale all input features.  

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

X = df.drop("price", axis=1).values
y = df["price"].values

X = StandardScaler().fit_transform(X)

5. Train a linear regressor on 70% of data (using inbuilt linear regression function of Python) and test its performance on remaining 30% of data. 

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

print("Linear Regression R²:", r2_score(y_test, y_pred))

Linear Regression R²: 0.8720694422097159


6. Reduce the dimensionality of the feature set using inbuilt PCA decomposition and then
again train a linear regressor on 70% of reduced data (using inbuilt linear regression
function of Python). Does it lead to any performance improvement on test set? 

In [25]:
pca = PCA(n_components=0.95)  # 95% variance
X_reduced = pca.fit_transform(X)

Xr_train, Xr_test, yr_train, yr_test = train_test_split(X_reduced, y, test_size=0.3, random_state=42)

reg2 = LinearRegression()
reg2.fit(Xr_train, yr_train)
yr_pred = reg2.predict(Xr_test)

print("After PCA R²:", r2_score(yr_test, yr_pred))


After PCA R²: 0.8602756046273354


PCA did not lead to performance improvement, since the R² score decreased slightly after dimensionality reduction.